**SENTIMENT ANALYSIS ON IMDB DATASET**

**NAME-GAURAV PRAKASH PAREEK**

I Have performed sentiment analysis on IMDB Dataset and used a RNN model with Biderctional LSTM.

In [ ]:
!wget https://www.dropbox.com/s/pdhwlpi2yeie0ol/movie-reviews-dataset.zip

--2021-08-05 12:54:13--  https://www.dropbox.com/s/pdhwlpi2yeie0ol/movie-reviews-dataset.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.18, 2620:100:6016:18::a27d:112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/pdhwlpi2yeie0ol/movie-reviews-dataset.zip [following]
--2021-08-05 12:54:13--  https://www.dropbox.com/s/raw/pdhwlpi2yeie0ol/movie-reviews-dataset.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc022849ff1e4acb9a12d87f57b3.dl.dropboxusercontent.com/cd/0/inline/BTpIfmDfY6uIrpbiQ0Gc8tggYKE_JgHnD_54Jgisi_UJbb1Q7Clamyc6Fd-sZbrILFIEiNIUxVxwlf9gm5U1EBG5odlP6Qr41RkKw3t2-ZiyF6G1TVkRleGaJFFr1YiLkhOTJhR6bE9IepsdGpD2NT2L/file# [following]
--2021-08-05 12:54:13--  https://uc022849ff1e4acb9a12d87f57b3.dl.dropboxusercontent.com/cd/0/inline/BTpIfmDfY6uIrpbiQ0Gc8tggYKE_JgHnD_54Jgisi_UJbb1Q7Cl

In [ ]:
!unzip -q "/content/movie-reviews-dataset.zip"

In [ ]:
from tensorflow.keras.preprocessing import  text_dataset_from_directory #used to load text data as tensorflow object
from tensorflow.strings import regex_replace    #fast regex replacement
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization #for coverting text to vector
from tensorflow.keras.models import Sequential 
from tensorflow.keras import Input
from tensorflow.keras.layers import Dense,RNN,LSTM,SimpleRNNCell,Embedding,Dropout,Bidirectional

In [ ]:
#takes input directory an return tensorflow data object
def prepareData(dir):
  data=text_dataset_from_directory(dir)
  return data.map(lambda text,label:(regex_replace(text,'<br />',' '),label),)

In [ ]:
train_data=prepareData('/content/movie-reviews-dataset/train')
test_data=prepareData('/content/movie-reviews-dataset/test')

for text_batch,label_batch in train_data.take(1):
  print(text_batch.numpy()[0])
  print(label_batch.numpy()[0])

Found 25000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.
b"Evidently lots of people really like this, but I found it infantilising and reasonably offensive codswallop, saved from oblivion by Jane Russell and a couple of memorable musical numbers, especially the opener (but there's a marked dip in invention later on). I don't get on with Monroe - she's supposed to be playing a dumb blonde who ain't that dumb, but she just comes over as dumb. Russell can't quite convince when she claims never to have been in a gymnasium, but is otherwise rather wonderful. The men are staggeringly uninteresting, as is the plot. By no means atrocious, but so patchy that, if this is a classic, God help us all."
0


In [ ]:
model=Sequential()
model.add(Input(shape=(1,),dtype="string"))

In [ ]:
max_tokens=1000
max_len=100
vectorize_layer=TextVectorization(max_tokens=max_tokens,
                                  output_mode="int",
                                  output_sequence_length=max_len
                                  )

In [ ]:
train_texts=train_data.map(lambda text,label:text)
vectorize_layer.adapt(train_texts)
model.add(vectorize_layer)

In [ ]:
model.add(Embedding(max_tokens + 1, 128)) #convert input  to fixed length
model.add(Bidirectional(LSTM(64)))
model.add(Dense(64,activation="relu"))
model.add(Dense(1,activation="sigmoid"))

In [ ]:
model.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])

In [ ]:
model.fit(train_data,epochs=10)

Epoch 1/10
782/782 [==============================] - 26s 22ms/step - loss: 0.5092 - accuracy: 0.7408
Epoch 2/10
782/782 [==============================] - 16s 21ms/step - loss: 0.4236 - accuracy: 0.8039
Epoch 3/10
782/782 [==============================] - 17s 22ms/step - loss: 0.3908 - accuracy: 0.8220
Epoch 4/10
782/782 [==============================] - 17s 21ms/step - loss: 0.3598 - accuracy: 0.8403
Epoch 5/10
782/782 [==============================] - 17s 22ms/step - loss: 0.3309 - accuracy: 0.8537
Epoch 6/10
782/782 [==============================] - 16s 21ms/step - loss: 0.2940 - accuracy: 0.8732
Epoch 7/10
782/782 [==============================] - 16s 21ms/step - loss: 0.2702 - accuracy: 0.8838
Epoch 8/10
782/782 [==============================] - 17s 22ms/step - loss: 0.2386 - accuracy: 0.8974
Epoch 9/10
782/782 [==============================] - 17s 21ms/step - loss: 0.2092 - accuracy: 0.9096
Epoch 10/10
782/782 [==============================] - 16s 21ms/step - loss: 0.194

In [ ]:
model.save('/content/drive/MyDrive/Colab Notebooks/review',save_format='tf')

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/review/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/review/assets


In [ ]:
from keras.models import load_model

In [ ]:
model=load_model('review')

In [ ]:
model.evaluate(test_data)

782/782 [==============================] - 12s 14ms/step - loss: 0.7806 - accuracy: 0.7600


[0.7806422114372253, 0.7600399851799011]

In [ ]:
text="This the worst movie I have ever watched"
model.predict([text])

array([[0.00173234]], dtype=float32)